# Code Explanation

## 1. Grab POI information from Google API

We are using `textsearch` function of Google Maps API. 
https://developers.google.com/maps/documentation/places/web-service/search-text

In [ ]:
import requests
import geopandas as gpd
from shapely.geometry import Point
import time

In [ ]:
# Formulate the query for collecting POI
lat = 40.1104
lng = -88.2273
search_item = 'Grocery Stores'
api_key = 

api_address = r"https://maps.googleapis.com/maps/api/place/textsearch/json?"
api_request = f"{api_address}&location={lat},{lng}&query={search_item}&key={api_key}"

print(api_request)

In [ ]:
# Collect POI from Google Maps API
api_response = requests.get(api_request).json()
print(api_response)

## 2. Retrieve results of Google Maps API

We can formalize the API return of Google Maps with json. There are multiple free online website. Here is <a href=https://jsonformatter.curiousconcept.com/#>one</a>. <br>

You will be realized that the return has attributes: `html_attributions`, `next_page_token`, `results` and `status`. If the results are valid, the `status` should be `OK`, and all the returns should be stored in `results`. As this API only returns 20 results, you can use `next_page_token` to retrieve more results (beyond the current 20). 

<img src="./data/api_result.jpg" style="width: 900px;"/>

The contents of `results` also have critical information about each POI, such as `business_status`, `geometry`, `name`, `operationg_hours`, `place_id`, so on. You can check the full list of attributes on <a href="https://developers.google.com/maps/documentation/places/web-service/search-text">this website</a>. <br>

For our application, we use `location` attributes, which consist of `lng` and `lat`, to convert the `results` into `GeoDataFrame`. 

In [ ]:
poi_gdf = gpd.GeoDataFrame(api_response['results'], crs='EPSG:4326')
        
poi_gdf['geometry'] = poi_gdf.apply(lambda x: Point(x['geometry']['location']['lng'],
                                                    x['geometry']['location']['lat']),
                                    axis=1
                                   )
poi_gdf

## 3. Obtain road network from OSM

In [ ]:
import osmnx as ox
import networkx as nx

In [ ]:
def remove_uncenessary_nodes(network):
    _nodes_removed = len([n for (n, deg) in network.out_degree() if deg == 0])
    network.remove_nodes_from([n for (n, deg) in network.out_degree() if deg == 0])
    for component in list(nx.strongly_connected_components(network)):
        if len(component) < 10:
            for node in component:
                _nodes_removed += 1
                network.remove_node(node)

    return network


def collect_road_network_from_OSM(location):
    network = ox.graph_from_place(location, network_type='drive', simplify=True)
    network = remove_uncenessary_nodes(network)
    
    return network

In [ ]:
location = 'Champaign County, IL'
G = collect_road_network_from_OSM(location)

ox.plot_graph(G)

In [ ]:
def find_nearest_osm(network, gdf):
    for idx, row in gdf.iterrows():
        if row.geometry.geom_type == 'Point':
            nearest_osm = ox.distance.nearest_nodes(network,
                                                    X=row.geometry.x,
                                                    Y=row.geometry.y
                                                    )
        elif row.geometry.geom_type == 'Polygon' or row.geometry.geom_type == 'MultiPolygon':
            nearest_osm = ox.distance.nearest_nodes(network,
                                                    X=row.geometry.centroid.x,
                                                    Y=row.geometry.centroid.y
                                                    )
        else:
            print(row.geometry.geom_type)
            continue

        gdf.at[idx, 'nearest_osm'] = nearest_osm

    return gdf


def calculate_service_area(gdf, network, dist):
    
    gdf = find_nearest_osm(network, gdf)

    nodes, edges = ox.graph_to_gdfs(network, nodes=True, edges=True, node_geometry=True)

    service_area = gpd.GeoDataFrame()
    for idx, row in gdf.iterrows():
        temp_nodes = nx.single_source_dijkstra_path_length(network, row['nearest_osm'], dist, weight='length')
        access_nodes = nodes.loc[nodes.index.isin(temp_nodes.keys()), 'geometry']
        temp_service_area = access_nodes.unary_union.convex_hull  # Create convex hull from the unioned nodes

        service_area.at[idx, 'geometry'] = temp_service_area

    service_area = service_area.set_crs(epsg=4326)

    return service_area


results_gdf = calculate_service_area(poi_gdf, G, 2000)
results_gdf.explore()

## 4. Implement more features of Google Maps API

Not only the locations from Google Maps API, but we can also incorporate the rating information into our application. 

In [ ]:
rating_val = 4.0
rating_count = 100

good_gdf = poi_gdf.loc[(poi_gdf['rating'] > rating_val) & (poi_gdf['user_ratings_total'] > rating_count)]
print(good_gdf.shape)
good_gdf

In [ ]:
results_gdf = calculate_service_area(good_gdf, G, 2000)
results_gdf.explore()

## 5. Create web map from `folium`

We utilize `folium` package to create a web map of our API query. For more information, please visit <a href=https://python-visualization.github.io/folium/>this website</a>. 

In [ ]:
import folium

In [ ]:
lat = 40.1104
lng = -88.2273

final_result = folium.Map([lat, lng], zoom_start=11, tiles='Stamen Toner')
final_result

In [ ]:
# Service area of Grocery stores
folium.GeoJson(results_gdf['geometry'].unary_union, # Locations of geometry
               style_function=lambda feature: { # Style
                   'fillColor': "green",
                   'color': "green",
                   'fillOpacity': 0.2,
               }).add_to(final_result) # Add to the existing web map

# Location of Grocery stores
folium.GeoJson(good_gdf['geometry'],  # Locations of geometry
               marker=folium.CircleMarker(radius=3,  # Radius in metres
                                          weight=0,  # outline weight
                                          fill_color='#006400',
                                          fill_opacity=1)
               ).add_to(final_result) # Add to the existing web map

final_result

## 6. Create interactive graphical user interface

To facilitate user's experience, Python has a package called <a href=https://ipywidgets.readthedocs.io/en/latest/>`ipywidgets`</a>. It will provide a graphical user interface so that non-programmers can interact with results generated from Python.

In [ ]:
import ipywidgets as widgets

In [ ]:
style = {'description_width': 'initial'}

w_loc = widgets.Text(
    value='Champaign County, IL',
    description='Location: ',
    style=style,
    disabled=False
)

w_poi = widgets.Text(
    value='Grocery Stores',
    description='Point of Interest: ',
    style=style,
    disabled=False
)

w_key = widgets.Text(
    value='',
    description='Google Maps API Key: ',
    style=style,
    disabled=False
)

w_dist = widgets.IntSlider(
    value=1000,
    min=500,
    max=3000,
    step=100,
    description='Max travel distance (in meters): ',
    style=style,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout=widgets.Layout(width='50%')
)

w_rate = widgets.FloatSlider(
    value=4,
    min=1,
    max=5,
    step=0.1,
    description='Minimum rating: ',
    style=style,
    readout=True,
    layout=widgets.Layout(width='50%')
)

w_count = widgets.IntSlider(
    value=10,
    min=1,
    max=100,
    step=1,
    description='Minimum review counts: ',
    style=style,
    readout=True,
    layout=widgets.Layout(width='50%')
)

display(w_loc)
display(w_poi)
display(w_key)
display(w_dist)
display(w_rate)
display(w_count)

button = widgets.Button(description="Submit")
output = widgets.Output()

display(button, output)

You can retrieve the input value of `ipywidgets` with the attribute `.value`.  

In [ ]:
location = w_loc.value
print(f'Location: {location}')

search_item = w_poi.value
print(f'Search Item: {search_item}')

api_key = w_key.value
print(f'API key: {api_key}')

distance = w_dist.value
print(f'Threshold distance: {distance}')

rating_min = w_rate.value
print(f'Minimun rating: {rating_min}')

rating_count_min = w_count.value
print(f'Minimum rating count: {rating_count_min}')

You can also define the function that is going to proceed with the button clicked. 

In [ ]:
def on_button_clicked(b):
    location = w_loc.value
    search_item = w_poi.value
    api_key = w_key.value
    distance = w_dist.value
    rating_min = w_rate.value
    rating_count_min = w_count.value

    with output:
        print(f"Your maximum travel distance to {search_item} is {distance} meters")
        print("Your accessible area is shown in green!")
        print("\n")

        loc = geocoder.google(location, key=api_key)
#             print(loc)

        latitude = loc.latlng[0]  # 40.115950
        longitude = loc.latlng[1]  # -88.241591

        print("#### STEP1: COLLECTING POI #### \n")
        poi_gdf = collect_point_of_interest(search_item, api_key, longitude, latitude)

        print("#### STEP2: COLLECTING ROAD NETWORK #### \n")
        G = collect_road_network_from_OSM(location)

        print("#### STEP3: CALCULATING SERVICE AREA #### \n")
        final_plot = show_results(poi_gdf, G, distance, rating_min, rating_count_min, longitude, latitude)

    display(final_plot)


button.on_click(on_button_clicked)

In [ ]:
style = {'description_width': 'initial'}

w_loc = widgets.Text(
    value='Champaign County, IL',
    description='Location: ',
    style=style,
    disabled=False
)

w_poi = widgets.Text(
    value='Grocery Stores',
    description='Point of Interest: ',
    style=style,
    disabled=False
)

w_key = widgets.Text(
    value='',
    description='Google Maps API Key: ',
    style=style,
    disabled=False
)

w_dist = widgets.IntSlider(
    value=1000,
    min=500,
    max=3000,
    step=100,
    description='Max travel distance (in meters): ',
    style=style,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    layout=widgets.Layout(width='50%')
)

w_rate = widgets.FloatSlider(
    value=4,
    min=1,
    max=5,
    step=0.1,
    description='Minimum rating: ',
    style=style,
    readout=True,
    layout=widgets.Layout(width='50%')
)

w_count = widgets.IntSlider(
    value=10,
    min=1,
    max=100,
    step=1,
    description='Minimum review counts: ',
    style=style,
    readout=True,
    layout=widgets.Layout(width='50%')
)

display(w_loc)
display(w_poi)
display(w_key)
display(w_dist)
display(w_rate)
display(w_count)

button = widgets.Button(description="Submit")
output = widgets.Output()

display(button, output)

button.on_click(on_button_clicked)